# ● XGBoost 모델 학습 준비

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report

In [2]:
# 데이터 로드
train_data = pd.read_csv('/home/encore/SKN07-2nd-3Team-main/data/churn-bigml-80.csv')
test_data = pd.read_csv('/home/encore/SKN07-2nd-3Team-main/data/churn-bigml-20.csv')

In [3]:
# 범주형 변수 인코딩
label_encoders = {}
categorical_columns = ['International plan', 'Voice mail plan', 'Churn']
for col in categorical_columns:
    le = LabelEncoder()
    train_data[col] = le.fit_transform(train_data[col])
    test_data[col] = le.transform(test_data[col])
    label_encoders[col] = le

# 수치형 변수 표준화
scaler = StandardScaler()
numerical_columns = [
    'Account length', 'Number vmail messages', 'Total day minutes', 'Total day calls',
    'Total day charge', 'Total eve minutes', 'Total eve calls', 'Total eve charge',
    'Total night minutes', 'Total night calls', 'Total night charge',
    'Total intl minutes', 'Total intl calls', 'Total intl charge',
    'Customer service calls'
]

train_data[numerical_columns] = scaler.fit_transform(train_data[numerical_columns])
test_data[numerical_columns] = scaler.transform(test_data[numerical_columns])

In [4]:
# 특성과 타겟 변수 분리
X_train = train_data.drop(columns=['Churn', 'State', 'Area code'], axis=1)
y_train = train_data['Churn']
X_test = test_data.drop(columns=['Churn', 'State', 'Area code'], axis=1)
y_test = test_data['Churn']

# ● 하이퍼파라미터 설정

## ○ GridSearchCV (그리드 탐색)

In [5]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

In [6]:
# XGBoost 모델 초기화
xgb_model = xgb.XGBClassifier()

In [7]:
# 하이퍼파라미터 그리드 정의
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],  # 학습률
    'max_depth': [3, 6, 9],  # 트리의 최대 깊이
    'n_estimators': [50, 100, 150],  # 트리의 개수
    'subsample': [0.8, 0.9, 1.0],  # 샘플링 비율
    'colsample_bytree': [0.8, 0.9, 1.0],  # 특성 샘플링 비율
    'min_child_weight': [1, 3, 5],  # 리프 노드의 최소 가중치 합
}

In [8]:
# GridSearchCV 객체 생성 (교차 검증 5번)
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, n_jobs=-1, scoring='accuracy')

In [ ]:
# 훈련 데이터로 학습
grid_search.fit(X_train, y_train)

# 최적의 하이퍼파라미터 출력
print("Best Hyperparameters:", grid_search.best_params_)

# 최적의 모델 평가
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}.")

## ○ RandomizedSearchCV (랜덤 탐색) 코드

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb
import numpy as np

In [ ]:
# XGBoost 모델 초기화
xgb_model = xgb.XGBClassifier()

In [ ]:
# 하이퍼파라미터 범위 정의
param_dist = {
    'learning_rate': np.arange(0.01, 0.2, 0.01),  # 학습률 범위
    'max_depth': [3, 6, 9, 12],  # 트리의 최대 깊이
    'n_estimators': [50, 100, 150, 200],  # 트리의 개수
    'subsample': np.arange(0.7, 1.0, 0.05),  # 샘플링 비율
    'colsample_bytree': np.arange(0.7, 1.0, 0.05),  # 특성 샘플링 비율
    'min_child_weight': [1, 3, 5, 7],  # 리프 노드의 최소 가중치 합
}

In [ ]:
# RandomizedSearchCV 객체 생성 (교차 검증 5번)
random_search = RandomizedSearchCV(estimator=xgb_model, param_distributions=param_dist, n_iter=50, cv=5, n_jobs=-1, scoring='accuracy')

In [ ]:
# 훈련 데이터로 학습
random_search.fit(X_train, y_train)

# 최적의 하이퍼파라미터 출력
print("Best Hyperparameters:", random_search.best_params_)

# 최적의 모델 평가
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# ● 모델 학습

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

# 최적 하이퍼파라미터로 모델 학습이 완료되었으므로, best_estimator_를 사용하여 예측을 시작합니다.
best_model = grid_search.best_estimator_  # 최적 모델 가져오기

# 모델 예측 (테스트 데이터에 대해 예측 수행)
y_pred = best_model.predict(X_test)  # 예측 값 생성

# 모델 성능 평가
accuracy = accuracy_score(y_test, y_pred)  # 정확도 계산
print(f"Accuracy: {accuracy:.4f}")  # 정확도 출력

# 더 상세한 평가 지표 (정밀도, 재현율, f1-score 등)
print("Classification Report:")
print(classification_report(y_test, y_pred))  # 분류 리포트 출력

# ● 예측 및 평가

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# 최적 모델 가져오기
best_model = grid_search.best_estimator_

# 테스트 데이터 예측
y_pred = best_model.predict(X_test)

# 정확도 출력
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# 분류 보고서 출력 (정밀도, 재현율, F1-score 등)
print("Classification Report:")
print(classification_report(y_test, y_pred))

# 혼동 행렬 계산
conf_matrix = confusion_matrix(y_test, y_pred)

# 혼동 행렬 시각화
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['No Churn', 'Churn'], yticklabels=['No Churn', 'Churn'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.tight_layout()
plt.show()

# ●  모델 튜닝

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report

# XGBoost 모델 초기화
xgb_model = xgb.XGBClassifier()

# 하이퍼파라미터 탐색 범위 (좁게 설정)
param_dist = {
    'learning_rate': [0.05, 0.1, 0.15],  # 학습률
    'max_depth': [4, 5, 6],  # 트리 최대 깊이
    'n_estimators': [100, 150, 200],  # 트리의 개수
    'subsample': [0.85, 0.9, 0.95],  # 샘플링 비율
    'colsample_bytree': [0.85, 0.9, 0.95],  # 특성 샘플링 비율
    'min_child_weight': [1, 2, 3],  # 리프 노드의 최소 가중치 합
    'gamma': [0, 0.1, 0.2],  # 손실 감소 최소값
}

# RandomizedSearchCV 사용 (랜덤으로 조합 탐색, 10번 반복)
random_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_dist,
    n_iter=10,  # 시도할 하이퍼파라미터 조합 수
    scoring='accuracy',  # 평가 기준
    cv=5,  # 교차 검증 수
    n_jobs=-1,  # 병렬 처리
    random_state=42  # 재현성을 위한 랜덤 시드
)

# 훈련 데이터로 학습
random_search.fit(X_train, y_train)

# 최적 하이퍼파라미터 및 성능 출력
print("Best Hyperparameters:", random_search.best_params_)

# 최적 모델 평가
best_tuned_model = random_search.best_estimator_
y_pred_tuned = best_tuned_model.predict(X_test)

# 평가 결과 출력
accuracy_tuned = accuracy_score(y_test, y_pred_tuned)
print(f"Accuracy after Tuning: {accuracy_tuned}")
print("\nClassification Report:\n", classification_report(y_test, y_pred_tuned))

# ● 피처 중요도 확인

## ○ 피처 중요도 추출 및 출력

In [ ]:
# 피처 중요도 추출
feature_importances = best_model.feature_importances_

# 중요도를 데이터프레임 형태로 정리
import pandas as pd
importance_df = pd.DataFrame({
    'Feature': X_train.columns,  # 특성 이름
    'Importance': feature_importances  # 중요도 값
}).sort_values(by='Importance', ascending=False)  # 중요도 기준으로 내림차순 정렬

# 중요도 데이터 출력
print(importance_df)

## ○ 피처 중요도 시각화

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 피처 중요도 시각화
plt.figure(figsize=(12, 8))
sns.barplot(x='Importance', y='Feature', data=importance_df, palette='viridis')
plt.title('Feature Importance', fontsize=16)
plt.xlabel('Importance', fontsize=12)
plt.ylabel('Feature', fontsize=12)
plt.tight_layout()
plt.show()

# ● 예측 정확도 확인

In [ ]:
from sklearn.metrics import accuracy_score

# 테스트 데이터에 대한 예측
y_pred = best_model.predict(X_test)

# 정확도 계산
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")